# Twitter Scraping for viral Tweets

As of now (October 2022), there is no way to retrieve the tweets from the Topic "viral tweets" from the Twitter API. Those are tweets that Twitter labels as viral based on number of likes, retweets and reaction surrounding the tweet in general. That's why we used online scraper tools to scrape tweets off this Twitter Topic page (namely https://twitter.com/i/topics/1284234742661963776).

The Viral Tweets topic is now disabled, instead we can use trending for you tabs data - which has the most trending hashtags - which in turn can be used to fetch viral tweets

In [1]:
DATA_PATH = "../../../data"
SCRAPED_TWEETS_PATH = f"{DATA_PATH}/scraped"

#### Octoparse

In [3]:
import os

OCTOPARSE = f"{SCRAPED_TWEETS_PATH}/octoparse/"
data_files = []

# iterate over files in
# that directory
for filename in os.listdir(OCTOPARSE):
    f = os.path.join(OCTOPARSE, filename)
    # checking if it is a file
    if os.path.isfile(f):
        data_files.append(f)

data_files

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../../../data/scraped/octoparse/'

In [ ]:
from bs4 import BeautifulSoup
import json

links = []

for data_file in data_files:
    with open(data_file, "r", encoding='utf8') as read_file:
        data = json.load(read_file)

        for field in data:
            for elem in field.values():
                soup = BeautifulSoup(elem)
                #print(soup.prettify())
                for link in soup.find_all("a"):
                    links.append(link.get("href"))  

octoparse_tweet_ids = set([link.split('/')[3] for link in links if 'status' in link])  

In [ ]:
len(octoparse_tweet_ids)

#### Other scraper

In [ ]:
import os

SAMPLE_SCRAPER = f"{SCRAPED_TWEETS_PATH}/sample-scraper/"
data_files = []

# iterate over files in
# that directory
for filename in os.listdir(SAMPLE_SCRAPER):
    f = os.path.join(SAMPLE_SCRAPER, filename)
    # checking if it is a file
    if os.path.isfile(f):
        data_files.append(f)

data_files

In [ ]:
links = []

for data_file in data_files:
    with open(data_file, "r", encoding='utf8') as read_file:
        data = json.load(read_file)

        for field in data.values():
            if type(field) == list:
                for elem in field:
                    links.append(elem['url'])

sample_scraper_tweet_ids = set([link.split('/')[5] for link in links if 'status' in link])

In [ ]:
len(sample_scraper_tweet_ids)

### Save all to csv

In [ ]:
viral_tweet_ids = list(octoparse_tweet_ids) + list(sample_scraper_tweet_ids)
len(viral_tweet_ids)

In [ ]:
# Save the ids into a file
import pandas as pd

viral_tweet_ids_df = pd.DataFrame(viral_tweet_ids, columns=['tweet_id'])
# Append csv
viral_tweet_ids_df.to_csv(f"{SCRAPED_TWEETS_PATH}/scraped_tweets_ids.csv", index=False, header=True)

snscrape

In [4]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

Search parameters

In [7]:
query = "ChatGPT"
min_likes = 20
language = "en"
since_year = "2023-8-01"
until_year = "2021-8-10"
xlsx_name = "tweets_aug_2023.xlsx"

In [8]:
tweets = []

search_query = f'{query} min_retweets:0 min_faves:{min_likes} lang:"{language}" since:{since_year} until:{until_year}'

for tweet in sntwitter.TwitterSearchScraper(search_query).get_items():
    # Extract relevant information from the tweet
    tweet_data = {
        "Id": tweet.id,
        "tweet": tweet.content,
        "Likes": tweet.likeCount,
        "Retweets": tweet.retweetCount,
        "Date": tweet.date.strftime("%Y-%m-%d %H:%M:%S"),
        "id": tweet.id,
        "created_at": tweet.date.strftime("%Y-%m-%d %H:%M:%S"),
        "tweet": tweet.rawContent,
        "replies_count": tweet.replyCount,
        "retweets_count": tweet.retweetCount,
        "likes_count": tweet.likeCount,
        "hashtags": tweet.hashtags,
        "cashtags": tweet.cashtags if tweet.cashtags else [],
        "language": tweet.lang,
        "link": tweet.url,
        "conversation_id": tweet.conversationId,
        "Jahr": tweet.date.year,
        "Monat": tweet.date.month,
        "Tag": tweet.date.day,
        "time": tweet.date.time().strftime("%H:%M:%S"),
        "user_id": tweet.user.id,
        "username": tweet.user.username,
    }
    
    tweets.append(tweet_data)

Error retrieving https://twitter.com/i/api/graphql/7jT5GT59P8IFjgxwqnEdQw/SearchTimeline?variables=%7B%22rawQuery%22%3A%22ChatGPT%20min_retweets%3A0%20min_faves%3A20%20lang%3A%5C%22en%5C%22%20since%3A2023-8-01%20until%3A2021-8-10%22%2C%22count%22%3A20%2C%22product%22%3A%22Latest%22%2C%22withDownvotePerspective%22%3Afalse%2C%22withReactionsMetadata%22%3Afalse%2C%22withReactionsPerspective%22%3Afalse%7D&features=%7B%22rweb_lists_timeline_redesign_enabled%22%3Afalse%2C%22blue_business_profile_image_shape_enabled%22%3Afalse%2C%22responsive_web_graphql_exclude_directive_enabled%22%3Atrue%2C%22verified_phone_label_enabled%22%3Afalse%2C%22creator_subscriptions_tweet_preview_api_enabled%22%3Afalse%2C%22responsive_web_graphql_timeline_navigation_enabled%22%3Atrue%2C%22responsive_web_graphql_skip_user_profile_image_extensions_enabled%22%3Afalse%2C%22tweetypie_unmention_optimization_enabled%22%3Atrue%2C%22vibe_api_enabled%22%3Atrue%2C%22responsive_web_edit_tweet_api_enabled%22%3Atrue%2C%22graphql

ScraperException: 4 requests to https://twitter.com/i/api/graphql/7jT5GT59P8IFjgxwqnEdQw/SearchTimeline?variables=%7B%22rawQuery%22%3A%22ChatGPT%20min_retweets%3A0%20min_faves%3A20%20lang%3A%5C%22en%5C%22%20since%3A2023-8-01%20until%3A2021-8-10%22%2C%22count%22%3A20%2C%22product%22%3A%22Latest%22%2C%22withDownvotePerspective%22%3Afalse%2C%22withReactionsMetadata%22%3Afalse%2C%22withReactionsPerspective%22%3Afalse%7D&features=%7B%22rweb_lists_timeline_redesign_enabled%22%3Afalse%2C%22blue_business_profile_image_shape_enabled%22%3Afalse%2C%22responsive_web_graphql_exclude_directive_enabled%22%3Atrue%2C%22verified_phone_label_enabled%22%3Afalse%2C%22creator_subscriptions_tweet_preview_api_enabled%22%3Afalse%2C%22responsive_web_graphql_timeline_navigation_enabled%22%3Atrue%2C%22responsive_web_graphql_skip_user_profile_image_extensions_enabled%22%3Afalse%2C%22tweetypie_unmention_optimization_enabled%22%3Atrue%2C%22vibe_api_enabled%22%3Atrue%2C%22responsive_web_edit_tweet_api_enabled%22%3Atr

In [9]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

# Set the search query and other parameters
query = "machine learning"
min_likes = 20
language = "en"
since_year = "2021-5-01" # Set starting month,year and day here #11 #1
until_year = "2021-5-10" # Set ending month,year and day #30 #31
csv_name = "tweets_july_2021.xlsx"
# Create a list to store the scraped tweets
tweets = []

# Define the search query
search_query = f'{query} min_retweets:0 min_faves:{min_likes} lang:"{language}" since:{since_year} until:{until_year}'

# Scrape tweets based on the search query
for tweet in sntwitter.TwitterSearchScraper(search_query).get_items():
  
    final_tweet = tweet
    tweet_data = {
        "Id":tweet.id,
        "tweet": tweet.rawContent,
        "Likes": tweet.likeCount,
        "Retweets": tweet.retweetCount,
        "Date": tweet.date.strftime("%Y-%m-%d %H:%M:%S"),
        "id": tweet.id,
        "created_at": tweet.date.strftime("%Y-%m-%d %H:%M:%S"),
        "tweet": tweet.rawContent,
        "negative": "",
        "neutral": "",
        "positive": "",
        "dominant": "",
        "replies_count": tweet.replyCount,
        "retweets_count": tweet.retweetCount,
        "likes_count": tweet.likeCount,
        "hashtags": tweet.hashtags,
        "cashtags": tweet.cashtags if tweet.cashtags else [] ,
        "language":tweet.lang,
        "link": tweet.url,
        "conversation_id": tweet.conversationId,
        "Jahr": tweet.date.year,
        "Monat": tweet.date.month,
        "Tag": tweet.date.day,
        "time": tweet.date.time().strftime("%H:%M:%S"),
        "user_id": tweet.user.id,
        "username": tweet.user.username,           
    }
    tweets.append(tweet_data)
# Create a pandas DataFrame from the scraped tweets
df = pd.DataFrame(tweets)
#Save data with name specified in csv_name 
df.to_excel(csv_name, index=False)

Error retrieving https://twitter.com/i/api/graphql/7jT5GT59P8IFjgxwqnEdQw/SearchTimeline?variables=%7B%22rawQuery%22%3A%22machine%20learning%20min_retweets%3A0%20min_faves%3A20%20lang%3A%5C%22en%5C%22%20since%3A2021-5-01%20until%3A2021-5-10%22%2C%22count%22%3A20%2C%22product%22%3A%22Latest%22%2C%22withDownvotePerspective%22%3Afalse%2C%22withReactionsMetadata%22%3Afalse%2C%22withReactionsPerspective%22%3Afalse%7D&features=%7B%22rweb_lists_timeline_redesign_enabled%22%3Afalse%2C%22blue_business_profile_image_shape_enabled%22%3Afalse%2C%22responsive_web_graphql_exclude_directive_enabled%22%3Atrue%2C%22verified_phone_label_enabled%22%3Afalse%2C%22creator_subscriptions_tweet_preview_api_enabled%22%3Afalse%2C%22responsive_web_graphql_timeline_navigation_enabled%22%3Atrue%2C%22responsive_web_graphql_skip_user_profile_image_extensions_enabled%22%3Afalse%2C%22tweetypie_unmention_optimization_enabled%22%3Atrue%2C%22vibe_api_enabled%22%3Atrue%2C%22responsive_web_edit_tweet_api_enabled%22%3Atrue%2

ScraperException: 4 requests to https://twitter.com/i/api/graphql/7jT5GT59P8IFjgxwqnEdQw/SearchTimeline?variables=%7B%22rawQuery%22%3A%22machine%20learning%20min_retweets%3A0%20min_faves%3A20%20lang%3A%5C%22en%5C%22%20since%3A2021-5-01%20until%3A2021-5-10%22%2C%22count%22%3A20%2C%22product%22%3A%22Latest%22%2C%22withDownvotePerspective%22%3Afalse%2C%22withReactionsMetadata%22%3Afalse%2C%22withReactionsPerspective%22%3Afalse%7D&features=%7B%22rweb_lists_timeline_redesign_enabled%22%3Afalse%2C%22blue_business_profile_image_shape_enabled%22%3Afalse%2C%22responsive_web_graphql_exclude_directive_enabled%22%3Atrue%2C%22verified_phone_label_enabled%22%3Afalse%2C%22creator_subscriptions_tweet_preview_api_enabled%22%3Afalse%2C%22responsive_web_graphql_timeline_navigation_enabled%22%3Atrue%2C%22responsive_web_graphql_skip_user_profile_image_extensions_enabled%22%3Afalse%2C%22tweetypie_unmention_optimization_enabled%22%3Atrue%2C%22vibe_api_enabled%22%3Atrue%2C%22responsive_web_edit_tweet_api_enab

Using tweepy to extract tweets

In [12]:
import os
os.chdir('D:\\Projects\\virality_detection_and_content_credibility_analysis\\spread-me-not')

import all libraries

In [20]:
import tweepy
import datetime
from collections import Counter
from heapq import nlargest
import yaml

load twitter a/c config info

In [21]:
with open('config/settings.yaml') as f:
    config = yaml.safe_load(f)

# Twitter API credentials (replace with your own)
consumer_key = config['twitter']['consumer_key']
consumer_secret = config['twitter']['consumer_secret']
access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']

### Authenticate with Twitter API

In [23]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True) #, wait_on_rate_limit_notify=True)

### Verify Authentication Status:

In [48]:
if auth.access_token is not None:
    print("Authentication successful!")
else:
    print("Authentication failed.")


Authentication successful!


### Make a simple API Request

In [58]:
import tweepy

# try:
#     user = api.me()
#     print(f"Connected to Twitter as {user.screen_name}")
# except TweepError as e:
#     print(f"Failed to fetch user information: {e}")



user = api.get_user(screen_name="your_twitter_screen_name")
print(f"Connected to Twitter as {user.screen_name}")
print(f"Name: {user.name}")
print(f"Followers: {user.followers_count}")
    # You can access other user properties as well
# except tweepy.TweepError as e:
#     print(f"Failed to fetch user information: {e}")

NotFound: 404 Not Found
50 - User not found.

### Define the topic and other parameters

In [40]:
topic = "rashmika"
min_retweet_count = 1
date_range_days = 10

# Calculate the start date for the date range
current_date = datetime.datetime.now()
start_date = current_date - datetime.timedelta(days=date_range_days)

# Fetch tweets using the Twitter API
tweets = tweepy.Cursor(api.search_tweets, q=topic, lang='en', tweet_mode='extended').items(1)

In [44]:
import json

rate_limit_status = api.rate_limit_status()
print(json.dumps(rate_limit_status, indent=4))

{
    "rate_limit_context": {
        "access_token": "1690909603368943617-2s6l4eOq5q2zt7RxsRMT2ng7Ed1CuM"
    },
    "resources": {
        "lists": {
            "/lists/list": {
                "limit": 15,
                "remaining": 15,
                "reset": 1692017851
            },
            "/lists/:id/tweets&GET": {
                "limit": 900,
                "remaining": 900,
                "reset": 1692017851
            },
            "/lists/:id/followers&GET": {
                "limit": 180,
                "remaining": 180,
                "reset": 1692017851
            },
            "/lists/memberships": {
                "limit": 75,
                "remaining": 75,
                "reset": 1692017851
            },
            "/lists/:id&DELETE": {
                "limit": 300,
                "remaining": 300,
                "reset": 1692017851
            },
            "/lists/subscriptions": {
                "limit": 15,
                "remaining": 

In [45]:
search_limits = rate_limit_status['resources']['search']['/search/tweets']
print(json.dumps(search_limits, indent=4))

{
    "limit": 180,
    "remaining": 180,
    "reset": 1692017851
}


In [46]:
api.retweet

<bound method API.retweet of <tweepy.api.API object at 0x00000196C363E1D0>>

In [41]:
tweets = []

for tweet in tweets:
    tweets.append(tweet)

In [42]:
tweets

[]

In [43]:
# Filter tweets based on retweet count and date range
filtered_tweets = [tweet for tweet in tweets if tweet.retweet_count >= min_retweet_count and tweet.created_at >= start_date]

# Calculate tweet frequency using Counter
tweet_text_list = [tweet.full_text for tweet in filtered_tweets]
tweet_frequency = Counter(tweet_text_list)

# Get the top N trending tweets (e.g., top 100)
top_n = 100
trending_tweets = nlargest(top_n, tweet_frequency, key=tweet_frequency.get)

# Print the trending tweets
for idx, tweet_text in enumerate(trending_tweets, start=1):
    print(f"{idx}. {tweet_text} - Retweets: {tweet_frequency[tweet_text]}")
